# Goals
1. Compare regions across Britain.

2. Find which confectionery has the largest and smallest profit margins regionally.

3. Identify which region peaks in sales over time.

4. Present the findings through:
    - Static visualizations (charts, graphs, etc.)
    - An interactive dashboard.



Revenue = Profit + Cost

Total Profit = Unit Profit * Unit Sold

Total Revenue = Revenue per Unit * Unit Sold

## 1. Import libraries, explore data

In [ ]:
# import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

pd.set_option('display.float_format', '{:,.2f}'.format)

In [ ]:
# load dataset
df = pd.read_excel('Confectionary [4564].xlsx')

# display first & last 5 rows
display(df.head())
print('')
display(df.tail())

In [ ]:
# check info & shape
df.info()
print('\nShape:', df.shape)

# check dupes
print('Duplicates:', df.duplicated().sum())

# check null values
print('\nNull values:')
display(df.isnull().sum())

In [ ]:
# statistical summary
df.describe().T

## 2. Preprocessing

In [ ]:
# create df_clean
df_clean = df.copy()

# sort values by date
df_clean = df_clean.sort_values(by='Date')

# rename columns
df_clean.rename(columns={'Revenue(£)': 'Total Revenue', 'Cost(£)': 'Total Cost', 'Profit(£)': 'Total Profit'}, inplace=True)

### 2.1 Feature Engineering

1. Create columns `Cost per Unit`, `Profit per Unit`, and `Revenue per Unit`
2. Calculate ratio to determine how much `Total Revenue` is scaled
3. Create column `Revenue per Unit_descaled` to show correct revenue per unit
4. Create column `Total Revenue_adjusted` to show correct total revenue
5. Standardize `Confectionary` column ('Caramel nut' == 'Caramel Nut' and 'Choclate Chunk' == 'Chocolate Chunk')

In [ ]:
# create columns to show cost, price, and revenue per unit
df_clean['Cost per Unit'] = df_clean['Total Cost'] / df_clean['Units Sold']
df_clean['Profit per Unit'] = df_clean['Total Profit'] / df_clean['Units Sold']
df_clean['Revenue per Unit'] = df_clean['Total Revenue'] / df_clean['Units Sold']

# statistical summary
df_clean[['Cost per Unit', 'Profit per Unit', 'Revenue per Unit']].describe().T

In [ ]:
# calculate ratio to show how many times larger `Revenue per Unit` is compared to cost and profit per unit
df_clean['ratio'] = df_clean['Revenue per Unit'] / (df_clean['Cost per Unit'] + df_clean['Profit per Unit'])
scaling_factor = df_clean['ratio'].mean()

display(df_clean.head())
print(f'\nRevenue per unit values are about {scaling_factor} times larger than cost and profit per unit values')

In [ ]:
# descale `Revenue per Unit` to show actual revenue per unit
df_clean['Revenue per Unit_descaled'] = df_clean['Cost per Unit'] + df_clean['Profit per Unit']

# adjust total revenue
df_clean['Total Revenue_adjusted'] = df_clean['Units Sold'] * df_clean['Revenue per Unit_descaled']

df_clean.head()

In [ ]:
df_clean[['Cost per Unit', 'Profit per Unit', 'Revenue per Unit_descaled']].describe().T

In [ ]:
# check unique `Confectionary` values
df_clean['Confectionary'].unique()

In [ ]:
# map variations to standard confectionary name
mapping = {
    'Caramel nut': 'Caramel Nut',
    'Choclate Chunk': 'Chocolate Chunk'
}

df_clean['Confectionary'] = df_clean['Confectionary'].replace(mapping)

In [ ]:
# check unique `Country(UK)` values
df_clean['Country(UK)'].unique()

### 2.4 `Units Sold`
1. Add `Total Profit` and `Total Cost` to get `Total Revenue`
2. Impute `Cost per Unit` and `Profit per Unit` by using the mean values for confectionary and region
3. Impute null Units Sold with `Total Cost` / `Cost per Unit`
4. Calculate `Revenue per Unit`

In [ ]:
# check nulls
df_clean[df_clean['Units Sold'].isnull()]

In [ ]:
# fill `Total Revenue` for rows where `Units Sold` is null
df_clean.loc[df_clean['Units Sold'].isna(), 'Total Revenue'] = (
    df_clean['Total Cost'] + df_clean['Total Profit']
)


df_clean[df_clean['Units Sold'].isnull()]

In [ ]:
# create subplot
fig, axes = plt.subplots(1, 2, figsize=(12, 6))
fig.suptitle('Cost and Profit per Unit Distribution by Confectionary', fontsize=16)

# boxplot for `Cost per Unit`
sns.boxplot(x='Confectionary', y='Cost per Unit', data=df_clean, ax=axes[0])
axes[0].set_title('Cost per Unit Distribution')
axes[0].set_ylabel('Cost per Unit (£)')
axes[0].set_xlabel('Confectionary')
axes[0].tick_params(axis='x', rotation=45)
axes[0].set_ylim(0, 3.5)

# boxplot for `Profit per Unit`
sns.boxplot(x='Confectionary', y='Profit per Unit', data=df_clean, ax=axes[1])
axes[1].set_title('Profit per Unit Distribution')
axes[1].set_ylabel('Profit per Unit (£)')
axes[1].set_xlabel('Confectionary')
axes[1].tick_params(axis='x', rotation=45)
axes[1].set_ylim(0, 3.5)

plt.tight_layout(rect=[0, 0, 1, 0.96])  # adjust spacing for suptitle
plt.show()

# summary table
confectionary_summary = df_clean.groupby('Confectionary').agg({
    'Cost per Unit': 'mean',
    'Profit per Unit': 'mean',
}).sort_values('Cost per Unit')
display(confectionary_summary)

In [ ]:
# filter data for Caramel in England
caramel_england = df_clean[(df_clean['Confectionary'] == 'Caramel') & (df_clean['Country(UK)'] == 'England')]

# create subplot
fig, axes = plt.subplots(1, 2, figsize=(14, 6))
fig.suptitle('Cost and Profit per Unit Distribution for Caramel in England', fontsize=16)

# histogram for `Cost per Unit`
sns.histplot(caramel_england['Cost per Unit'].dropna(), bins=5, kde=False, ax=axes[0])
axes[0].set_title('Cost per Unit')
axes[0].set_xlabel('Cost per Unit (£)')
axes[0].set_ylabel('Frequency')
axes[0].set_ylim(0, 65)


# histogram for `Profit per Unit`
sns.histplot(caramel_england['Profit per Unit'].dropna(), bins=5, kde=False, ax=axes[1])
axes[1].set_title('Profit per Unit')
axes[1].set_xlabel('Profit per Unit (£)')
axes[1].set_ylabel('Frequency')
axes[1].set_ylim(0, 65)


plt.tight_layout(rect=[0, 0, 1, 0.95])
plt.show()

In [ ]:
mask = df_clean['Units Sold'].isna()

# mean `Cost per Unit` per Confectionary and Region
cost_mean = df_clean.groupby(['Confectionary', 'Country(UK)'])['Cost per Unit'].transform('mean')
profit_mean = df_clean.groupby(['Confectionary', 'Country(UK)'])['Profit per Unit'].transform('mean')

# fill only where `Units Sold` == null
df_clean.loc[mask, 'Cost per Unit'] = cost_mean[mask]
df_clean.loc[mask, 'Profit per Unit'] = profit_mean[mask]

df_clean[df_clean['Units Sold'].isnull()]

In [ ]:
# fill `Units Sold` nulls by calculating `Total Cost` / `Cost per Unit`
df_clean.loc[mask, 'Units Sold'] = df_clean.loc[mask, 'Total Cost'] / df_clean.loc[mask, 'Cost per Unit']
df_clean[df_clean['Revenue per Unit'].isnull()]

In [ ]:
# fill missing values for `Revenue per Unit`
df_clean['Revenue per Unit'] = df_clean['Total Revenue'] / df_clean['Units Sold']

In [ ]:
# descale `Revenue per Unit` to show actual revenue per unit
df_clean['Revenue per Unit_descaled'] = df_clean['Cost per Unit'] + df_clean['Profit per Unit']

# adjust total revenue
df_clean['Total Revenue_adjusted'] = df_clean['Units Sold'] * df_clean['Revenue per Unit_descaled']


### 2.2 `Total Cost` & `Cost per Unit`

1. Impute null values for `Cost per Unit` with mean
2. Impute null values for `Total Cost` by multiplying `Cost per Unit` with `Units Sold`

In [ ]:
# check nulls
df_clean[df_clean['Total Cost'].isnull()]

In [ ]:
# caramel is the only confectionary with variation in `Cost per Unit`
# as caramel is not one of the confectionaries with missing cost values, imput `Cost per Unit` nulls with mean for each confectionary
df_clean['Cost per Unit'] = df_clean.groupby('Confectionary')['Cost per Unit'].transform(lambda x: x.fillna(x.mean()))

# check null values in `Total Cost`
df_clean[df_clean['Total Cost'].isnull()]

In [ ]:
# fill null values in `Total Cost`
df_clean['Total Cost'] = (df_clean['Cost per Unit'] * df_clean['Units Sold'])

### 2.3 `Total Profit` & `Profit per Unit`
1. Impute nulls for `Profit per Unit` when `Confectionary` == 'Caramel'
2. Impute remaining nulls for `Profit per Unit`
3. Impute nulls for `Profit` by multiplying `Profit per Unit` with `Units Sold`

In [ ]:
# check nulls
df_clean[df_clean['Total Profit'].isnull()]

In [ ]:
# as caramel has variation in both cost and profit per unit, impute this separately
# the caramel missing profit values has `Country(UK)` == 'England' and `Cost per Unit` == 0.20
# compare all caramel products with these same values
df_clean[(df_clean['Confectionary'] == 'Caramel') & (df_clean['Cost per Unit'] == 0.20) & (df_clean['Country(UK)'] == 'England')]

In [ ]:
# mask for `Confectionary` == caramel, `Cost per Unit` == 0.20 and `Profit per Unit` == null
mask = (df_clean['Confectionary'] == 'Caramel') & (df_clean['Cost per Unit'] == 0.20) & (df_clean['Profit per Unit'].isna())

# calculate mean
mean_profit_per_unit = df_clean.loc[(df_clean['Confectionary'] == 'Caramel') & (df_clean['Cost per Unit'] == 0.20), 'Profit per Unit'].mean()

# impute null values for rows in mask
df_clean.loc[mask, 'Profit per Unit'] = mean_profit_per_unit

In [ ]:
# check remaining nulls
df_clean[df_clean['Total Profit'].isnull()]

In [ ]:
# impute `Profit per Unit` with mean for remaining rows
df_clean['Profit per Unit'] = df_clean.groupby('Confectionary')['Profit per Unit'].transform(lambda x: x.fillna(x.mean()))

In [ ]:
# fill null values in `Total Profit`
df_clean['Total Profit'] = (df_clean['Profit per Unit'] * df_clean['Units Sold'])

### 2.5 Final Feature Engineering
1. Drop unnecessary columns (`Total Revenue`, `Revenue per Unit`, `ratio`)
2. Fix remaining null values for new columns
2. Rename columns (`Revenue per Unit_descaled` = Revenue per Unit, `Total Revenue_adjusted` = Total Revenue)
3. Ensure `Date` column is datetime


In [ ]:
# drop unnecessary columns
df_clean.drop(columns=['Total Revenue', 'Revenue per Unit', 'ratio'], inplace=True)

# recalculate `Revenue per Unit_descaled` and `Total Revenue_adjusted` columns to remove nulls
df_clean['Revenue per Unit_descaled'] = df_clean['Cost per Unit'] + df_clean['Profit per Unit']
df_clean['Total Revenue_adjusted'] = df_clean['Units Sold'] * df_clean['Revenue per Unit_descaled']

In [ ]:
# rename columns
df_clean.rename(columns={'Revenue per Unit_descaled': 'Revenue per Unit', 'Total Revenue_adjusted': 'Total Revenue'}, inplace=True)

In [ ]:
# ensure `Date` is datetime
df_clean['Date'] = pd.to_datetime(df_clean['Date'])

In [ ]:
df_clean.head()
df_clean.isnull().sum()

## 3. EDA

In [ ]:
# distribution of units sold for each confectionary
plt.figure(figsize=(10,6))
sns.boxplot(x='Confectionary', y='Units Sold', data=df_clean)
plt.title('Units Sold Distribution by Confectionary')
plt.ylabel('Units Sold')

In [ ]:
cat_cols = ['Country(UK)', 'Confectionary']
color_discrete_sequence = px.colors.qualitative.Safe


for col in cat_cols:
    # get counts for sorting
    counts = df_clean[col].value_counts().reset_index()
    counts.columns = [col, 'count']

    # bar chart
    fig = px.bar(
        counts,
        x='count',
        y=col,
        orientation='h',
        text='count',
        color=col,
        color_discrete_sequence=px.colors.qualitative.Safe,
        title=f'Number of {col} Observations'
    )

    fig.update_layout(
        yaxis={'categoryorder':'total ascending'},
        template='plotly_white'
    )

    # display text on bars
    fig.update_traces(textposition='outside', texttemplate='%{text}')

    fig.show()

In [ ]:
# columns to plot
cols = ['Total Revenue', 'Total Profit', 'Total Cost', 'Units Sold']

# subplot
fig, axes = plt.subplots(2, 2, figsize=(14, 10))
fig.suptitle('Distributions of Revenue, Profit, Cost, and Units Sold', fontsize=16)

# flatten axes array
axes = axes.flatten()

# loop through columns and axes
for ax, col in zip(axes, cols):
    sns.histplot(df_clean[col], bins=50, kde=True, ax=ax)
    ax.set_title(col)
    ax.set_xlabel(col if col != 'Units Sold' else 'Units Sold')
    ax.set_ylabel('Frequency')
    ax.set_ylim(0, 85)

plt.tight_layout(rect=[0, 0, 1, 0.96])
plt.show()

## 4. Regional comparison of sales and profits

In [ ]:
# confectionary comparison
regional_comparison = df_clean.groupby(['Country(UK)']).agg({
    'Total Revenue': 'sum',
    'Total Profit': 'sum',
    'Units Sold': 'sum'
}).reset_index()

display(regional_comparison)

In [ ]:
# stacked bar chart for regional comparison of sales and profits

# confectionary comparison
regional_confectionary_comparison = df_clean.groupby(['Country(UK)', 'Confectionary']).agg({
    'Total Revenue': 'sum',
    'Total Profit': 'sum',
    'Units Sold': 'sum'
}).reset_index()

region_totals = regional_confectionary_comparison.groupby('Country(UK)')[['Total Revenue', 'Total Profit']].sum().reset_index()

color_discrete_sequence = px.colors.qualitative.Safe

# find max value for both metrics
y_max = max(regional_confectionary_comparison["Total Revenue"].max(),
            regional_confectionary_comparison["Total Profit"].max())

# stacked bar chart for Total Revenue
fig = px.bar(
    regional_confectionary_comparison,
    x='Country(UK)',
    y='Total Revenue',
    color='Confectionary',
    title='Total Revenue by Region and Confectionery (Stacked)',
    labels={'Total Revenue': 'Total Revenue (£)', 'Country(UK)': 'Region'},
    color_discrete_sequence=px.colors.qualitative.Safe,
    barmode='stack',
    category_orders={'Country(UK)': regional_confectionary_comparison.groupby('Country(UK)')['Total Revenue'].sum().sort_values(ascending=False).index}
)

# add total revenue labels on top of bars
for i, row in region_totals.iterrows():
    fig.add_annotation(
        x=row['Country(UK)'],
        y=row['Total Revenue'],
        text=f"£{row['Total Revenue']:,.0f}",
        showarrow=False,
        font=dict(color='black', size=12, family='Arial'),
        yshift=10
    )

fig.update_yaxes(range=[0, 1700000])
fig.update_layout(xaxis_title='Region', yaxis_title='Total Revenue (£)', legend_title_text='Confectionery')
fig.show()

# stacked bar chart for Total Profit
fig = px.bar(
    regional_confectionary_comparison,
    x='Country(UK)',
    y='Total Profit',
    color='Confectionary',
    title='Total Profit by Region and Confectionery (Stacked)',
    labels={'Total Profit': 'Total Profit (£)', 'Country(UK)': 'Region'},
    color_discrete_sequence=px.colors.qualitative.Safe,
    barmode='stack',
    category_orders={'Country(UK)': regional_confectionary_comparison.groupby('Country(UK)')['Total Profit'].sum().sort_values(ascending=False).index}

)

# add total profit labels on top of bars
for i, row in region_totals.iterrows():
    fig.add_annotation(
        x=row['Country(UK)'],
        y=row['Total Profit'],
        text=f"£{row['Total Profit']:,.0f}",
        showarrow=False,
        font=dict(color='black', size=12, family='Arial'),
        yshift=10
    )

fig.update_yaxes(range=[0, 1700000])
fig.update_layout(xaxis_title='Region', yaxis_title='Total Profit (£)', legend_title_text='Confectionery')
fig.show()

In [ ]:
# 100% stacked bar chart to show % of confectionary contribution per region

# calculate the proportion (as %)
regional_confectionary_comparison['Revenue %'] = regional_confectionary_comparison.groupby('Country(UK)')['Total Revenue'].transform(lambda x: x / x.sum() * 100)
regional_confectionary_comparison['Profit %'] = regional_confectionary_comparison.groupby('Country(UK)')['Total Profit'].transform(lambda x: x / x.sum() * 100)

color_discrete_sequence = px.colors.qualitative.Safe

# 100% stacked bar chart for Total Revenue
fig = px.bar(
    regional_confectionary_comparison,
    x='Country(UK)',
    y='Revenue %',
    color='Confectionary',
    title='Revenue Share by Region and Confectionery (100% Stacked)',
    labels={'Revenue %': 'Revenue Share (%)', 'Country(UK)': 'Region'},
    color_discrete_sequence=px.colors.qualitative.Safe,
    barmode='stack',
    category_orders={'Country(UK)': regional_confectionary_comparison.groupby('Country(UK)')['Total Revenue'].sum().sort_values(ascending=False).index}
)

fig.update_layout(
    yaxis=dict(title='Revenue Share (%)', range=[0, 100]),
    xaxis_title='Region',
    legend_title_text='Confectionery',
    template='plotly_white'
)
fig.show()

# 100% stacked bar chart for Total Profit
fig = px.bar(
    regional_confectionary_comparison,
    x='Country(UK)',
    y='Profit %',
    color='Confectionary',
    title='Profit Share by Region and Confectionery (100% Stacked)',
    labels={'Profit %': 'Profit Share (%)', 'Country(UK)': 'Region'},
    color_discrete_sequence=px.colors.qualitative.Safe,
    barmode='stack',
    category_orders={'Country(UK)': regional_confectionary_comparison.groupby('Country(UK)')['Total Profit'].sum().sort_values(ascending=False).index}
)

fig.update_layout(
    yaxis=dict(title='Profit Share (%)', range=[0, 100]),
    xaxis_title='Region',
    legend_title_text='Confectionery',
    template='plotly_white'
)


fig.show()

In [ ]:
# confectionary profit & revenue pivot tables
pivot_confectionary_profit = df_clean.pivot_table(index="Country(UK)", columns="Confectionary", values='Total Profit',
                              aggfunc="sum", fill_value=0)
pivot_confectionary_profit['Total'] = pivot_confectionary_profit.sum(axis=1)
pivot_confectionary_profit = pivot_confectionary_profit.sort_values(by='Total', ascending=False)



pivot_confectionary_revenue = df_clean.pivot_table(index="Country(UK)", columns="Confectionary", values='Total Revenue',
                              aggfunc="sum", fill_value=0)
pivot_confectionary_revenue['Total'] = pivot_confectionary_revenue.sum(axis=1)
pivot_confectionary_revenue = pivot_confectionary_revenue.sort_values(by='Total', ascending=False)

print('Confectionary Profit by Region')
display(pivot_confectionary_profit)
print('\nConfectionary Revenue by Region')
display(pivot_confectionary_revenue)

In [ ]:
# unique regions
unique_regions = df_clean['Country(UK)'].unique()

for region in unique_regions:
    # filter data for current region
    region_data = df_clean[df_clean['Country(UK)'] == region]

    # aggregate by confectionary
    agg_data = region_data.groupby('Confectionary', as_index=False)[['Total Revenue', 'Total Profit', 'Units Sold']].sum()

    # bubble chart
    fig = px.scatter(
        agg_data,
        x='Total Revenue',
        y='Total Profit',
        size='Units Sold',
        color='Confectionary',
        color_discrete_sequence=px.colors.qualitative.Safe,
        hover_data={
            'Total Revenue': ':,.0f',
            'Total Profit': ':,.0f',
            'Units Sold': ':,.0f'
        },
        title=f'Revenue vs Profit by Confectionary in {region}',
        size_max=60
    )

    fig.update_layout(
        template='plotly_white',
        xaxis_title='Revenue (£)',
        yaxis_title='Profit (£)'
    )

    fig.update_xaxes(range=[0, 800000])
    fig.update_yaxes(range=[0, 500000])

    fig.show()



## 5. Highest and lowest profit margins

In [ ]:
# calculate profit margin
df_clean['Profit Margin'] = df_clean['Total Profit'] / df_clean['Total Revenue'] * 100

# aggregate average profit margin per confectionary per region
margin_region = df_clean.groupby(['Country(UK)', 'Confectionary'], as_index=False)['Profit Margin'].mean()

# calculate overall average margin per confectionary for ordering
order = margin_region.groupby('Confectionary')['Profit Margin'].mean().sort_values(ascending=False).index

color_discrete_sequence = px.colors.qualitative.Safe

# grouped bar chart
fig = px.bar(
    margin_region,
    x='Country(UK)',
    y='Profit Margin',
    color='Confectionary',
    color_discrete_sequence=px.colors.qualitative.Safe,
    barmode='group',
    category_orders={'Confectionary': order},
    title='Average Profit Margin per Confectionary Across Regions',
    labels={'Profit Margin': 'Profit Margin (%)', 'Confectionary': 'Confectionary', 'Country(UK)': 'Region'}
)

#fig.update_yaxes(tickformat='.%')
fig.show()

In [ ]:
# calculate total revenue and profit per confectionary
profit_margin_confectionary = (
    df_clean.groupby('Confectionary')
    .agg({'Total Revenue': 'sum', 'Total Profit': 'sum'})
    .reset_index()
)

# compute Profit Margin (%)
profit_margin_confectionary['Profit Margin (%)'] = (
    profit_margin_confectionary['Total Profit'] / profit_margin_confectionary['Total Revenue']
) * 100

display(profit_margin_confectionary.sort_values('Profit Margin (%)', ascending=False))

In [ ]:
# sort df by profit margin descending
df_sorted = profit_margin_confectionary.sort_values(
    by='Profit Margin (%)',
    ascending=False
)

fig = px.bar(
    df_sorted,
    x='Confectionary',
    y='Profit Margin (%)',
    text='Profit Margin (%)',
    title='Profit Margin (%) by Confectionary',
    color='Profit Margin (%)',
    color_continuous_scale='viridis'
)

fig.update_traces(texttemplate='%{text:.1f}%', textposition='outside')
fig.update_layout(
    template='plotly_white',
    xaxis_title='Confectionary',
    yaxis_title='Profit Margin (%)',
    coloraxis_colorbar_title='Margin %'
)

fig.show()


In [ ]:
# average profit margin per confectionary per region
profit_margin_region = (
    df_clean.groupby(['Country(UK)', 'Confectionary'], as_index=False)['Profit Margin']
    .mean()
)

# unique regions
regions = profit_margin_region['Country(UK)'].unique()

# loop through regions
for region in regions:

    # sort descending for the current region
    df_region_sorted = (
        profit_margin_region[profit_margin_region['Country(UK)'] == region]
        .sort_values(by='Profit Margin', ascending=False)
    )

    # bar chart for this region
    fig = px.bar(
        df_region_sorted,
        x='Confectionary',
        y='Profit Margin',
        text='Profit Margin',
        title=f'Profit Margin by Confectionary — {region}',
        color='Profit Margin',
        color_continuous_scale='viridis'   # color-blind-safe gradient
    )

    fig.update_traces(texttemplate='%{text:.1f}%', textposition='outside')
    fig.update_layout(
        template='plotly_white',
        xaxis_title='Confectionary',
        yaxis_title='Profit Margin (%)',
        coloraxis_colorbar_title='Margin %',
        yaxis=dict(range=[0, profit_margin_region['Profit Margin'].max() * 1.1])
    )

    fig.show()


## 6. Time-based sales trends

In [ ]:
import plotly.graph_objects as go

# unique regions
regions = df_clean['Country(UK)'].unique()

# loop through each region to create a separate chart
for region in regions:
    # filter data for the current region
    region_df = df_clean[df_clean['Country(UK)'] == region]

    # aggregate monthly revenue for the current region
    region_monthly_revenue = region_df.groupby(pd.Grouper(key='Date', freq='MS'))['Total Revenue'].sum().reset_index()

    # find peak revenue for the current region
    peak_sales_region = region_monthly_revenue.loc[region_monthly_revenue['Total Revenue'].idxmax()]
    peak_month_year = peak_sales_region['Date'].strftime('%b %Y')

    # create a line chart for the region's monthly revenue
    fig = px.line(
        region_monthly_revenue,
        x='Date',
        y='Total Revenue',
        title=f'Monthly Revenue Trend for {region}',
        labels={'Total Revenue': 'Revenue (£)', 'Date': 'Month'}
    )

    # add marker for the peak
    fig.add_trace(
        go.Scatter(
            x=[peak_sales_region['Date']],
            y=[peak_sales_region['Total Revenue']],
            mode='markers+text',
            marker=dict(color='red', size=10),
            text=[f"Peak: £{peak_sales_region['Total Revenue']:,.0f}<br>Date: {peak_month_year}"],
            textposition='top center',
            name=f"{region} Peak",
            showlegend=False
        )
    )

    fig.update_layout(
        xaxis_title='Date',
        yaxis_title='Total Revenue (£)',
        xaxis=dict(range=['2000-01-01','2005-12-31']),
        template='plotly_white',
        yaxis_range=[0, 89000]
    )

    fig.show()

In [ ]:
# calculate average monthly revenue per region and standard deviation
monthly_revenue_region = df_clean.groupby([pd.Grouper(key='Date', freq='MS'), 'Country(UK)'])['Total Revenue'].sum().reset_index()
monthly_revenue_region = monthly_revenue_region.groupby('Country(UK)')['Total Revenue'].agg(['mean', 'std']).sort_values(by='mean', ascending=False).reset_index()
monthly_revenue_region

In [ ]:
# aggregate total revenue by month and region
sales_over_time_m = (
    df_clean.groupby([pd.Grouper(key='Date', freq='MS'), 'Country(UK)'])['Total Revenue']
    .sum()
    .reset_index()
    .sort_values('Date')
)

# extract Year and Month name for animation and axis
sales_over_time_m['Year'] = sales_over_time_m['Date'].dt.year
sales_over_time_m['Month'] = sales_over_time_m['Date'].dt.strftime('%b')

# ensure correct month
month_order = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun',
               'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

sales_over_time_m['Month'] = pd.Categorical(
    sales_over_time_m['Month'], categories=month_order, ordered=True
)

# animated area chart
fig = px.area(
    sales_over_time_m,
    x='Month',
    y='Total Revenue',
    color='Country(UK)',
    animation_frame='Year',
    color_discrete_sequence=px.colors.qualitative.Safe,
    title='Monthly Total Revenue by Region (Animated by Year)',
    labels={'Total Revenue': 'Total Revenue (£)', 'Country(UK)': 'Region'},
    category_orders={'Month': month_order}
)

fig.update_layout(
    template='plotly_white',
    hovermode='x unified',
    xaxis_title='Month',
    yaxis_title='Total Revenue (£)',
    legend_title_text='Region'
)

fig.layout.updatemenus[0].buttons[0].args[1]['frame']['duration'] = 1500  # ms per frame

fig.show()

## 7. Dashboard

In [ ]:
#!pip install plotly ipywidgets --quiet

from google.colab import output
output.enable_custom_widget_manager()

import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import ipywidgets as widgets
from IPython.display import display, clear_output
import plotly.io as pio

pio.renderers.default = "colab"


In [ ]:
# Widgets
description_width = '150px'

year_slider = widgets.IntSlider(value=2000, min=2000, max=2005, step=1,
                                description="Year", style={'description_width': description_width})
freq_dropdown = widgets.Dropdown(options=["Monthly","Quarterly"], value="Monthly",
                                 description="Frequency", style={'description_width': description_width})
region_dropdown = widgets.Dropdown(options=["All"] + sorted(df_clean["Country(UK)"].unique().tolist()),
                                   value="All", description="Region", style={'description_width': description_width})
confectionary_dropdown = widgets.Dropdown(options=["All"] + sorted(df_clean["Confectionary"].unique().tolist()),
                                          value="All", description="Confectionary", style={'description_width': description_width})
output_area = widgets.Output()

freq_map = {"Monthly":"MS","Quarterly":"QS"}

In [ ]:
# custom title
def build_title(year, region, confectionary):
    suffix = ""
    if region != "All":
        suffix += f" | {region}"
    if confectionary != "All":
        suffix += f" | {confectionary}"
    return f"Sales Dashboard — {year}{suffix}"

In [ ]:
# update dashboard function
def update_dashboard(change):
    with output_area:
        clear_output(wait=True)
        year = year_slider.value
        freq_label = freq_dropdown.value
        freq_code = freq_map[freq_label]
        region = region_dropdown.value
        conf = confectionary_dropdown.value

        # filter dataset
        df_year = df_clean[df_clean["Date"].dt.year == year].copy()
        if region != "All":
            df_year = df_year[df_year["Country(UK)"] == region]
        if conf != "All":
            df_year = df_year[df_year["Confectionary"] == conf]

        # date range
        date_range = pd.date_range(start=f"{year}-01-01", end=f"{year}-12-31", freq=freq_code)

        # aggregates
        temp_rev = df_year.set_index("Date").resample(freq_code)["Total Revenue"].sum().reindex(date_range, fill_value=0).reset_index()
        temp_rev.rename(columns={"index":"Date"}, inplace=True)
        temp_prof = df_year.set_index("Date").resample(freq_code)["Total Profit"].sum().reindex(date_range, fill_value=0).reset_index()
        temp_prof.rename(columns={"index":"Date"}, inplace=True)
        temp_cost = df_year.set_index("Date").resample(freq_code)["Total Cost"].sum().reindex(date_range, fill_value=0).reset_index()
        temp_cost.rename(columns={"index":"Date"}, inplace=True)

        profit_margin = (temp_prof["Total Profit"] / temp_rev["Total Revenue"]).replace([float('inf'), -float('inf')], 0).fillna(0)

        # hover formatting
        if freq_label=="Monthly":
            hover_fmt="%b %Y"; dtick="M1"; tickformat="%b"
        elif freq_label=="Quarterly":
            hover_fmt="Q%q %Y"; dtick="M3"; tickformat="Q%q"
        else:
            hover_fmt="%Y"; dtick="M12"; tickformat="%Y"

        # shared max for y-axis
        shared_max = max(temp_rev["Total Revenue"].max(), temp_prof["Total Profit"].max(), temp_cost["Total Cost"].max(), 1) * 1.1
        pm_max = max(profit_margin.max()*100, 1) * 1.1

        # subplots
        fig = make_subplots(
            rows=2, cols=2,
            specs=[[{"type":"scatter"},{"type":"bar"}],
                   [{"colspan":2,"type":"scatter"}, None]],
            subplot_titles=("Revenue", "Profit & Cost", "Profit Margin (%)")
        )

        # revenue area
        fig.add_trace(go.Scatter(
            x=temp_rev["Date"], y=temp_rev["Total Revenue"],
            mode="lines", fill="tozeroy", line=dict(color="blue", width=2),
            hovertemplate="Date: %{x|"+hover_fmt+"}<br>Revenue: £%{y:,.0f}<extra></extra>",
            showlegend=False
        ), row=1, col=1)

        # profit & cost stacked bars
        fig.add_trace(go.Bar(
            x=temp_cost["Date"], y=temp_cost["Total Cost"], name="Cost", marker_color="red",
            hovertemplate="Cost: £%{y:,.0f}<br>Date: %{x|"+hover_fmt+"}<extra></extra>"
        ), row=1, col=2)
        fig.add_trace(go.Bar(
            x=temp_prof["Date"], y=temp_prof["Total Profit"], name="Profit", marker_color="blue",
            hovertemplate="Profit: £%{y:,.0f}<br>Date: %{x|"+hover_fmt+"}<extra></extra>"
        ), row=1, col=2)

        # profit margin
        fig.add_trace(go.Scatter(
            x=temp_rev["Date"], y=profit_margin*100,
            mode="lines+markers", line=dict(color="orange", width=2),
            hovertemplate="Profit Margin: %{y:.1f}%<br>Date: %{x|"+hover_fmt+"}<extra></extra>",
            showlegend=False
        ), row=2, col=1)

        # axes
        fig.update_yaxes(range=[0, shared_max], row=1, col=1)
        fig.update_yaxes(range=[0, shared_max], row=1, col=2)
        fig.update_yaxes(range=[0, pm_max], row=2, col=1, title_text="Profit Margin (%)")
        for r,c in [(1,1),(1,2),(2,1)]:
            fig.update_xaxes(type="date", dtick=dtick, tickformat=tickformat, row=r, col=c)

        fig.update_layout(
            template="plotly_white",
            autosize=False, width=1200, height=700,
            margin=dict(l=40,r=40,t=100,b=60),
            barmode="stack",
            showlegend=True,
            title_text=build_title(year, region, conf),
            title_x=0.5
        )

        display(fig)


In [ ]:
# observe widgets
for w in [year_slider, freq_dropdown, region_dropdown, confectionary_dropdown]:
    w.observe(lambda change: update_dashboard(None), names="value")

# arrange filters horizontally
filter_row = widgets.HBox([region_dropdown, confectionary_dropdown, freq_dropdown, year_slider])

# display filters and output area
display(filter_row, output_area)
update_dashboard(None)